# Recommender Code Along

The classic recommender tutorial uses the [movielens data set](https://grouplens.org/datasets/movielens/). It is similar to using the iris or MNIST data set for other algorithms. Let's do a code along to get an idea of how this all works!


Looking for more datasets? Check out: https://gist.github.com/entaroadun/1653794

In [17]:
from pyspark.sql import SparkSession

In [18]:
spark = SparkSession.builder.appName('rec').getOrCreate()

With Collaborative filtering we make predictions (filtering) about the interests of a user by collecting preferences or taste information from many users (collaborating). The underlying assumption is that if a user A has the same opinion as a user B on an issue, A is more likely to have B's opinion on a different issue x than to have the opinion on x of a user chosen randomly.

The image below (from Wikipedia) shows an example of collaborative filtering. At first, people rate different items (like videos, images, games). Then, the system makes predictions about a user's rating for an item not rated yet. The new predictions are built upon the existing ratings of other users with similar ratings with the active user. In the image, the system predicts that the user will not like the video.

<img src=https://upload.wikimedia.org/wikipedia/commons/5/52/Collaborative_filtering.gif />

Spark MLlib library for Machine Learning provides a Collaborative Filtering implementation by using Alternating Least Squares. The implementation in MLlib has these parameters:

* numBlocks is the number of blocks used to parallelize computation (set to -1 to auto-configure).
* rank is the number of latent factors in the model.
* iterations is the number of iterations to run.
* lambda specifies the regularization parameter in ALS.
* implicitPrefs specifies whether to use the explicit feedback ALS variant or one adapted for implicit feedback data.
* alpha is a parameter applicable to the implicit feedback variant of ALS that governs the baseline confidence in preference observations.

Let's see this all in action!

In [19]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS

In [20]:
data = spark.read.csv('movielens_ratings.csv',inferSchema=True,header=True)

In [21]:
data.head()

Row(movieId=2, rating=3.0, userId=0)

In [22]:
data.show()

+-------+------+------+
|movieId|rating|userId|
+-------+------+------+
|      2|   3.0|     0|
|      3|   1.0|     0|
|      5|   2.0|     0|
|      9|   4.0|     0|
|     11|   1.0|     0|
|     12|   2.0|     0|
|     15|   1.0|     0|
|     17|   1.0|     0|
|     19|   1.0|     0|
|     21|   1.0|     0|
|     23|   1.0|     0|
|     26|   3.0|     0|
|     27|   1.0|     0|
|     28|   1.0|     0|
|     29|   1.0|     0|
|     30|   1.0|     0|
|     31|   1.0|     0|
|     34|   1.0|     0|
|     37|   1.0|     0|
|     41|   2.0|     0|
+-------+------+------+
only showing top 20 rows



In [23]:
data.describe().show()

+-------+------------------+------------------+------------------+
|summary|           movieId|            rating|            userId|
+-------+------------------+------------------+------------------+
|  count|              1501|              1501|              1501|
|   mean| 49.40572951365756|1.7741505662891406|14.383744170552964|
| stddev|28.937034065088994| 1.187276166124803| 8.591040424293272|
|    min|                 0|               1.0|                 0|
|    max|                99|               5.0|                29|
+-------+------------------+------------------+------------------+



We can do a split to evaluate how well our model performed, but keep in mind that it is very hard to know conclusively how well a recommender system is truly working for some topics. Especially if subjectivity is involved, for example not everyone that loves star wars is going to love star trek, even though a recommendation system may suggest otherwise.

In [24]:
# Smaller dataset so we will use 0.8 / 0.2
(training, test) = data.randomSplit([0.8, 0.2])

In [25]:
# Build the recommendation model using Alternating Least Squares on the training data
als = ALS(maxIter=5, regParam=0.01, userCol="userId", itemCol="movieId", ratingCol="rating")
model = als.fit(training)

Now let's see hwo the model performed!

In [26]:
# Evaluate the model by computing the RMSE on the test data
# Root mean square error
predictions = model.transform(test)

In [27]:
predictions.show()

+-------+------+------+------------+
|movieId|rating|userId|  prediction|
+-------+------+------+------------+
|     31|   1.0|    26|   -2.715691|
|     31|   1.0|    13|  0.49450773|
|     31|   1.0|    19|   2.1619642|
|     31|   1.0|     4| -0.26832715|
|     85|   1.0|    12|    2.267573|
|     85|   3.0|     6|-0.106328726|
|     85|   2.0|    20|  0.51573634|
|     85|   1.0|     5|   1.6654606|
|     85|   1.0|     4|   3.1510124|
|     85|   4.0|     7|   3.4048965|
|     65|   2.0|    15|   1.8015229|
|     65|   1.0|     2|  0.79052836|
|     53|   1.0|    12|   -2.991323|
|     53|   2.0|    19|   1.2324266|
|     53|   1.0|    23|   1.4752154|
|     78|   1.0|    13|  0.63751113|
|     34|   3.0|     3|  -1.7534913|
|     34|   1.0|    19|-0.046562195|
|     34|   1.0|    17|   -2.162646|
|     34|   1.0|     4|    3.056128|
+-------+------+------+------------+
only showing top 20 rows



In [28]:
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating",predictionCol="prediction")
rmse = evaluator.evaluate(predictions)
print("Root-mean-square error = " + str(rmse))

Root-mean-square error = 1.71703288463


The RMSE described our error in terms of the stars rating column.

So now that we have the model, how would you actually supply a recommendation to a user?

The same way we did with the test data! For example:

In [29]:
single_user = test.filter(test['userId']==11).select(['movieId','userId'])

In [30]:
# User had 10 ratings in the test data set 
# Realistically this should be some sort of hold out set!
single_user.show()

+-------+------+
|movieId|userId|
+-------+------+
|      0|    11|
|      6|    11|
|     16|    11|
|     18|    11|
|     19|    11|
|     21|    11|
|     30|    11|
|     32|    11|
|     37|    11|
|     38|    11|
|     51|    11|
|     71|    11|
|     72|    11|
|     90|    11|
|     94|    11|
|     97|    11|
+-------+------+



In [31]:
reccomendations = model.transform(single_user)

In [32]:
reccomendations.orderBy('prediction',ascending=False).show()

+-------+------+-----------+
|movieId|userId| prediction|
+-------+------+-----------+
|     94|    11|  6.9679346|
|     90|    11|  5.1912837|
|     18|    11|  4.6447606|
|     19|    11|   4.472289|
|     37|    11|  3.6165526|
|     71|    11|   2.406548|
|      0|    11|  2.0288494|
|     30|    11|  1.8286213|
|     72|    11|  1.7775459|
|     32|    11|  1.7494404|
|     38|    11|   1.682413|
|      6|    11|   1.536159|
|     97|    11|  1.3400635|
|     16|    11|  0.5144206|
|     21|    11| -0.1412406|
|     51|    11|-0.81785786|
+-------+------+-----------+



# Great Job!